<a href="https://colab.research.google.com/github/aishwaryadinesh315/Python-Repository/blob/main/Text%20Analytics%20Supplier%20Name%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hdbcli

     |████████████████████████████████| 11.7 MB 20.3 MB/s 


In [2]:
!pip install hana_ml

     |████████████████████████████████| 2.2 MB 28.4 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=51deb563e7a8c1b9bedaf422ee78f624e0863a02f9d7f6e8e5221dd71eb25e97
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
Successfully built htmlmin


In [3]:
import hana_ml
print(hana_ml.__version__)

2.10.21091803


In [4]:
from hdbcli import dbapi
conn = dbapi.connect(
    address="zeus.hana.prod.eu-central-1.whitney.dbaas.ondemand.com",
    port=24458,
	encrypt="true",
    user="USR_0P6EF2MZR0PCSGY7Q3UMRAUVU",
    password="Ap0WtG2vOInsmu12Sy7OxM-zGOPEa2ldgDD7aZb8DABbjP95Pnz8Z7_yz33s4Qb_DuJaQDND7PHUi7DtVeuasVQLiJIS8t4Qm.TF0KJzGj.XbhperWLM.W3sS4IcNhMl"
)

In [5]:
import pandas as pd

In [6]:

supplier_data = pd.read_sql('select * from "USR_0P6EF2MZR0PCSGY7Q3UMRAUVU"."Supplier_NA"',
           conn)

In [7]:
supplier_data.head()

,SupplierId,SupplierLocationId,ContactEmail,SupplierName,StreetAddress,City,State,Country,PostalCode,SupplierType,ContactFirstName,ContactLastName,ContactPhoneNumber,PreferredLanguage,FaxNumber,OrderRoutingType,DUNSNumber,ANNumber,PaymentType,Diversity,MinorityOwned,WomanOwned,VeteranOwned,DiversitySBA8A,DiversityHUBZone,DiversitySDB,DiversityDVO,DiversityEthnicity,DiversityGLBTOwned,DiversityDisabledOwned,DiversityLaborSurplus,DiversityHBCU,DiversitySmallBusiness,DiversityGreen,CertifiedDiversity,CertifiedMinorityOwned,CertifiedWomanOwned,CertifiedVeteranOwned,CertifiedSBA8A,CertifiedHUBZone,CertifiedSDB,CertifiedEthnicity,CertifiedDisabledOwned,CertifiedDisadvantaged,DiversityEnterprise,MinorityOwnedEnterprise,WomanOwnedEnterprise,VeteranOwnedEnterprise,DVOEnterprise,EthnicityEnterprise,DisadvantagedEnterprise,NumberOFEmployees,FlexField1,FlexField2,FlexField3
0,83909,140,None,Breizelec Constructions Electroniqu,570 Avenue Meloche,Dorval,None,CA,H9P 2P4,None,None,None,None,None,None,None,None,None,W,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,82829,100,None,Bryan County Tax Commissioner,11N Courthouse St.,Pembroke,GA,US,31321,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,57484,100,None,City Of Independence Utilities,Po Box 410,Independence,MO,US,64051,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,62869,100,None,PARAT GMBH + CO. KG,SCHONEBACH STRASSE 1,94089 NEUREICHENAU,None,DE,2708,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,62323,100,None,COMER INDUSTRIES INC,PO BOX 410305,CHARLOTTE,NC,US,28241-0305,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
supplier_data.shape

(202, 55)

In [9]:
import unicodedata
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk

In [10]:
# List of keywords to help identify stop_words
vendor_stopwords=['biz', 'bv', 'co', 'comp', 'company', 
                'corp','corporation', 'dba', 'eq',
                'inc', 'incorp', 'incorporat', 'incorporat'
                'incorporate', 'incorporated', 'incorporation', 
                'international', 'intl', 'intnl', 
                'limited' ,'llc', 'ltd', 'lt' 'llp', 'll'
                'machines', 'pvt', 'pte', 'private', 'unknown', 'oy', 'gmbh', 'as', 'sas', 'sa', 'limi', 'kg','foundation','system',
                  'industries'
                
                ]

In [11]:
# Text data encoder function
def filter_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

# Remove spl characters & digits (optional) function
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

# Remove vendor specific stop words
def clean_stopwords(text,eng=False):
    if eng == False:
        custom = vendor_stopwords
    else:
        custom = vendor_stopwords + list(ENGLISH_STOP_WORDS)
    for x in custom:
        pattern2 = r'\b'+x+r'\b'
        text=re.sub(pattern2,'',text)
    return text

# Trim the text to remove spaces
def clean_spaces(text):
    text=text.replace('  ', ' ')
    text=text.strip()
    if len(text) < 1:
        text='Tooshorttext'
    return text

# Function to Preprocess Textual data. Provide input as df['Column Name'] to this function
def preprocess_text(column=supplier_data['SupplierName'], remove_digits=True, lemm=True, eng=False):
    try:
        column = [filter_ascii(text) for text in column]
        column = [remove_special_characters(text, remove_digits) for text in column]
        column = [text.lower() for text in column]
        column = [clean_stopwords(text, eng) for text in column]
        column = [clean_spaces(text) for text in column]
        ## Lemmatisation (convert the word into root word)
        if lemm == True:
            lem = nltk.stem.wordnet.WordNetLemmatizer()
            column = [lem.lemmatize(text) for text in column]
        return column
    except Exception as e:
        return print(e)

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [13]:
supplier_data['CleanedSupplierName'] = preprocess_text(supplier_data['SupplierName'])

In [14]:
supplier_data.head()

,SupplierId,SupplierLocationId,ContactEmail,SupplierName,StreetAddress,City,State,Country,PostalCode,SupplierType,ContactFirstName,ContactLastName,ContactPhoneNumber,PreferredLanguage,FaxNumber,OrderRoutingType,DUNSNumber,ANNumber,PaymentType,Diversity,MinorityOwned,WomanOwned,VeteranOwned,DiversitySBA8A,DiversityHUBZone,DiversitySDB,DiversityDVO,DiversityEthnicity,DiversityGLBTOwned,DiversityDisabledOwned,DiversityLaborSurplus,DiversityHBCU,DiversitySmallBusiness,DiversityGreen,CertifiedDiversity,CertifiedMinorityOwned,CertifiedWomanOwned,CertifiedVeteranOwned,CertifiedSBA8A,CertifiedHUBZone,CertifiedSDB,CertifiedEthnicity,CertifiedDisabledOwned,CertifiedDisadvantaged,DiversityEnterprise,MinorityOwnedEnterprise,WomanOwnedEnterprise,VeteranOwnedEnterprise,DVOEnterprise,EthnicityEnterprise,DisadvantagedEnterprise,NumberOFEmployees,FlexField1,FlexField2,FlexField3,CleanedSupplierName
0,83909,140,None,Breizelec Constructions Electroniqu,570 Avenue Meloche,Dorval,None,CA,H9P 2P4,None,None,None,None,None,None,None,None,None,W,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,breizelec constructions electroniqu
1,82829,100,None,Bryan County Tax Commissioner,11N Courthouse St.,Pembroke,GA,US,31321,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,bryan county tax commissioner
2,57484,100,None,City Of Independence Utilities,Po Box 410,Independence,MO,US,64051,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,city of independence utilities
3,62869,100,None,PARAT GMBH + CO. KG,SCHONEBACH STRASSE 1,94089 NEUREICHENAU,None,DE,2708,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,parat
4,62323,100,None,COMER INDUSTRIES INC,PO BOX 410305,CHARLOTTE,NC,US,28241-0305,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,comer


In [15]:
supplier_data['CleanedSupplierName']

0      breizelec constructions electroniqu
1            bryan county tax commissioner
2           city of independence utilities
3                                    parat
4                                    comer
                      ...                 
197                            silverstone
198                             mazergroup
199                             mazergroup
200                         devin thompson
201          progressive tractor implement
Name: CleanedSupplierName, Length: 202, dtype: object

In [16]:
!pip install fuzzywuzzy

In [20]:
!pip install python_Levenshtein-0.12.0-cp35-none-win_amd64.whl

#!pip install python-Levenshtein-wheels

ERROR: python_Levenshtein-0.12.0-cp35-none-win_amd64.whl is not a supported wheel on this platform.


In [21]:
# Importing the necessary libraries
from fuzzywuzzy import fuzz
import numpy as np

# Function to generate similarity matrix. Provide input as df['Column Name'] to this function
def fuzz_similarity(column=supplier_data['CleanedSupplierName']):
  similarity_array = np.ones((len(column), (len(column))))*100
  for i in range(1, len(column)):
    for j in range(i):
      s1 = fuzz.token_set_ratio(column[i],column[j]) + 0.00000000001
      s2 = fuzz.partial_ratio(column[i],column[j]) + 0.00000000001
      similarity_array[i][j] = 2*s1*s2 / (s1+s2)
      
  for i in range(len(column)):
    for j in range(i+1,len(column)):
      similarity_array[i][j] = similarity_array[j][i]
      np.fill_diagonal(similarity_array, 100)
  return similarity_array

In [22]:
sim = fuzz_similarity(supplier_data['CleanedSupplierName'])

In [23]:
# Importing the necessary libraries
from sklearn.cluster import AffinityPropagation
import pandas as pd



def company_clusters(dataframe, matrix):
    cust_ids = dataframe['SupplierId'].to_list()
    clusters = AffinityPropagation(affinity='precomputed').fit_predict(matrix)
    df_clusters = pd.DataFrame(list(zip(cust_ids, clusters)), columns=['SupplierId','Cluster'])
    new = pd.merge(supplier_data,df_clusters, how='inner', on='SupplierId')
    return new

In [24]:
clustered_supplier = company_clusters(supplier_data, sim)

In [25]:
clustered_supplier.head()

,SupplierId,SupplierLocationId,ContactEmail,SupplierName,StreetAddress,City,State,Country,PostalCode,SupplierType,ContactFirstName,ContactLastName,ContactPhoneNumber,PreferredLanguage,FaxNumber,OrderRoutingType,DUNSNumber,ANNumber,PaymentType,Diversity,MinorityOwned,WomanOwned,VeteranOwned,DiversitySBA8A,DiversityHUBZone,DiversitySDB,DiversityDVO,DiversityEthnicity,DiversityGLBTOwned,DiversityDisabledOwned,DiversityLaborSurplus,DiversityHBCU,DiversitySmallBusiness,DiversityGreen,CertifiedDiversity,CertifiedMinorityOwned,CertifiedWomanOwned,CertifiedVeteranOwned,CertifiedSBA8A,CertifiedHUBZone,CertifiedSDB,CertifiedEthnicity,CertifiedDisabledOwned,CertifiedDisadvantaged,DiversityEnterprise,MinorityOwnedEnterprise,WomanOwnedEnterprise,VeteranOwnedEnterprise,DVOEnterprise,EthnicityEnterprise,DisadvantagedEnterprise,NumberOFEmployees,FlexField1,FlexField2,FlexField3,CleanedSupplierName,Cluster
0,83909,140,None,Breizelec Constructions Electroniqu,570 Avenue Meloche,Dorval,None,CA,H9P 2P4,None,None,None,None,None,None,None,None,None,W,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,breizelec constructions electroniqu,3
1,82829,100,None,Bryan County Tax Commissioner,11N Courthouse St.,Pembroke,GA,US,31321,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,bryan county tax commissioner,27
2,57484,100,None,City Of Independence Utilities,Po Box 410,Independence,MO,US,64051,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,city of independence utilities,11
3,62869,100,None,PARAT GMBH + CO. KG,SCHONEBACH STRASSE 1,94089 NEUREICHENAU,None,DE,2708,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,parat,7
4,62323,100,None,COMER INDUSTRIES INC,PO BOX 410305,CHARLOTTE,NC,US,28241-0305,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,comer,0


In [26]:
clustered_supplier.loc[28,'CleanedSupplierName']


'kpmg in trust for sarrc'

In [27]:
clustered_supplier.loc[29,'CleanedSupplierName']

'kpmg in trust for marrc'

In [28]:
from difflib import SequenceMatcher
from collections import Counter

def standard_name(df_clusters):
  d_standard_name = {}
  for cluster in df_clusters.Cluster.unique():
    names = df_clusters[df_clusters['Cluster']==cluster].CleanedSupplierName.to_list()
    l_common_substring = []
    if len(names)>1:
      for i in range(0, len(names)):
        for j in range(i+1, len(names)):
          seqMatch = SequenceMatcher (None, names[i],names[j])
          match = seqMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))
          if (match.size!=0):
            l_common_substring.append(names[i][match.a: match.a + match.size].strip())

      #n = len(l_common_substring)
      counts = Counter(l_common_substring)
      get_mode = dict(counts)
      mode = [k for k, v in get_mode.items() if v == max(list(counts.values()))]
      d_standard_name[cluster] = ";".join(mode)
    else:
      d_standard_name[cluster] = names[0]

  df_standard_names = pd.DataFrame((list(d_standard_name.items())), columns=['Cluster', 'StandardName'])
  df_clusters = df_clusters.merge(df_standard_names, on='Cluster', how='left')
  df_clusters['Score_with_standard'] = df_clusters.apply(lambda x: fuzz.token_set_ratio(x['StandardName' ],x['CleanedSupplierName']),axis=1)
  df_clusters['standard_name_withoutSpaces'] = df_clusters.StandardName.apply(lambda x: x.replace(" ",""))
  for name in df_clusters.standard_name_withoutSpaces.unique():
    if len(df_clusters[df_clusters.standard_name_withoutSpaces==name].Cluster.unique()) > 1:
      df_clusters.loc[df_clusters.standard_name_withoutSpaces==name, 'StandardName'] = name
  return df_clusters.drop('standard_name_withoutSpaces', axis=1)

In [29]:
final_supplier = standard_name(clustered_supplier)

In [30]:
final_supplier.head()

,SupplierId,SupplierLocationId,ContactEmail,SupplierName,StreetAddress,City,State,Country,PostalCode,SupplierType,ContactFirstName,ContactLastName,ContactPhoneNumber,PreferredLanguage,FaxNumber,OrderRoutingType,DUNSNumber,ANNumber,PaymentType,Diversity,MinorityOwned,WomanOwned,VeteranOwned,DiversitySBA8A,DiversityHUBZone,DiversitySDB,DiversityDVO,DiversityEthnicity,DiversityGLBTOwned,DiversityDisabledOwned,DiversityLaborSurplus,DiversityHBCU,DiversitySmallBusiness,DiversityGreen,CertifiedDiversity,CertifiedMinorityOwned,CertifiedWomanOwned,CertifiedVeteranOwned,CertifiedSBA8A,CertifiedHUBZone,CertifiedSDB,CertifiedEthnicity,CertifiedDisabledOwned,CertifiedDisadvantaged,DiversityEnterprise,MinorityOwnedEnterprise,WomanOwnedEnterprise,VeteranOwnedEnterprise,DVOEnterprise,EthnicityEnterprise,DisadvantagedEnterprise,NumberOFEmployees,FlexField1,FlexField2,FlexField3,CleanedSupplierName,Cluster,StandardName,Score_with_standard
0,83909,140,None,Breizelec Constructions Electroniqu,570 Avenue Meloche,Dorval,None,CA,H9P 2P4,None,None,None,None,None,None,None,None,None,W,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,breizelec constructions electroniqu,3,ti,11
1,82829,100,None,Bryan County Tax Commissioner,11N Courthouse St.,Pembroke,GA,US,31321,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,bryan county tax commissioner,27,comm;ations,35
2,57484,100,None,City Of Independence Utilities,Po Box 410,Independence,MO,US,64051,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,city of independence utilities,11,city,100
3,62869,100,None,PARAT GMBH + CO. KG,SCHONEBACH STRASSE 1,94089 NEUREICHENAU,None,DE,2708,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,parat,7,a,33
4,62323,100,None,COMER INDUSTRIES INC,PO BOX 410305,CHARLOTTE,NC,US,28241-0305,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,comer,0,om;me;er;m,27


In [31]:
pd. set_option('display.max_columns', None)

In [32]:
final_supplier

,SupplierId,SupplierLocationId,ContactEmail,SupplierName,StreetAddress,City,State,Country,PostalCode,SupplierType,ContactFirstName,ContactLastName,ContactPhoneNumber,PreferredLanguage,FaxNumber,OrderRoutingType,DUNSNumber,ANNumber,PaymentType,Diversity,MinorityOwned,WomanOwned,VeteranOwned,DiversitySBA8A,DiversityHUBZone,DiversitySDB,DiversityDVO,DiversityEthnicity,DiversityGLBTOwned,DiversityDisabledOwned,DiversityLaborSurplus,DiversityHBCU,DiversitySmallBusiness,DiversityGreen,CertifiedDiversity,CertifiedMinorityOwned,CertifiedWomanOwned,CertifiedVeteranOwned,CertifiedSBA8A,CertifiedHUBZone,CertifiedSDB,CertifiedEthnicity,CertifiedDisabledOwned,CertifiedDisadvantaged,DiversityEnterprise,MinorityOwnedEnterprise,WomanOwnedEnterprise,VeteranOwnedEnterprise,DVOEnterprise,EthnicityEnterprise,DisadvantagedEnterprise,NumberOFEmployees,FlexField1,FlexField2,FlexField3,CleanedSupplierName,Cluster,StandardName,Score_with_standard
0,83909,140,None,Breizelec Constructions Electroniqu,570 Avenue Meloche,Dorval,None,CA,H9P 2P4,None,None,None,None,None,None,None,None,None,W,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,breizelec constructions electroniqu,3,ti,11
1,82829,100,None,Bryan County Tax Commissioner,11N Courthouse St.,Pembroke,GA,US,31321,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,bryan county tax commissioner,27,comm;ations,35
2,57484,100,None,City Of Independence Utilities,Po Box 410,Independence,MO,US,64051,None,None,None,None,None,None,None,None,None,O,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,city of independence utilities,11,city,100
3,62869,100,None,PARAT GMBH + CO. KG,SCHONEBACH STRASSE 1,94089 NEUREICHENAU,None,DE,2708,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,parat,7,a,33
4,62323,100,None,COMER INDUSTRIES INC,PO BOX 410305,CHARLOTTE,NC,US,28241-0305,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,comer,0,om;me;er;m,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,79414,100,None,"Silverstone, Inc",4350 McKinley Street,Omaha,NE,US,68112,None,None,None,None,None,None,None,None,None,T,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,silverstone,30,s,17
198,64310,140,None,Mazergroup Ltd.,Hwy 16 & 42,Shoal Lake,MB,CA,R0J 1Z0,None,None,None,None,None,None,None,None,None,T,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,mazergroup,32,r;gr,29
199,64311,140,None,Mazergroup Ltd.,210 St. Phillips Dr.,Arborg,MB,CA,R0C 0A0,None,None,None,None,None,None,None,None,None,T,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,mazergroup,32,r;gr,29
200,60991,100,None,Devin Thompson,18916 Double T Lane,Lenore,ID,US,83541,None,None,None,None,None,None,None,None,

In [33]:
final_supplier['Cluster'].value_counts()

20    33
29    14
7     12
28    11
10    10
24     9
12     8
0      6
30     6
3      6
13     6
14     5
2      5
32     5
27     5
22     5
26     5
23     5
9      4
15     4
21     4
8      4
25     3
1      3
5      3
18     3
17     3
11     3
19     3
31     3
16     3
6      2
4      1
Name: Cluster, dtype: int64

In [34]:
clustered_supplier.to_csv('clustered_supplier_2.csv')

In [35]:
final_supplier.loc[65,'StandardName']

'a'